In [7]:
import polars as pl
csv_file = '../data/yellow_tripdata_2023-02.csv'
parquet_file = '../data/yellow_tripdata_2023-02.parquet'

In [8]:
df = pl.read_parquet(parquet_file)
df.head(2)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2023-02-01 00:32:53,2023-02-01 00:34:34,2,0.3,1,"""N""",142,163,2,4.4,3.5,0.5,0.0,0.0,1.0,9.4,2.5,0.0
2,2023-02-01 00:35:16,2023-02-01 00:35:30,1,0.0,1,"""N""",71,71,4,-3.0,-1.0,-0.5,0.0,0.0,-1.0,-5.5,0.0,0.0


# Utilizing Column Expression
- using gt() with select
- using eq() with select
- calculating total time travelled
- calculating total distance travelled in km
- finding min, max of trip_distance

In [9]:
(
    df
    .select([
        'VendorID',
        'trip_distance',
        pl.col('trip_distance').gt(0).alias('dist_greater_than_zero')
    ])
    .head()
)

VendorID,trip_distance,dist_greater_than_zero
i32,f64,bool
1,0.3,true
2,0.0,false
2,0.0,false
1,18.8,true
2,3.22,true


In [10]:
(
    df
    .select([
        'VendorID',
        'trip_distance',
        pl.col('trip_distance').eq(0).alias('dist_equals_zero')
    ])
    .head()
)

VendorID,trip_distance,dist_equals_zero
i32,f64,bool
1,0.3,false
2,0.0,true
2,0.0,true
1,18.8,false
2,3.22,false


In [11]:
(
    df
    .select([
        'VendorID',
        'tpep_dropoff_datetime',
        'tpep_pickup_datetime',
        (pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).alias('total_duration_of_travel')
    ])
    .head()
)

VendorID,tpep_dropoff_datetime,tpep_pickup_datetime,total_duration_of_travel
i32,datetime[ns],datetime[ns],duration[ns]
1,2023-02-01 00:34:34,2023-02-01 00:32:53,1m 41s
2,2023-02-01 00:35:30,2023-02-01 00:35:16,14s
2,2023-02-01 00:35:30,2023-02-01 00:35:16,14s
1,2023-02-01 01:01:38,2023-02-01 00:29:33,32m 5s
2,2023-02-01 00:25:46,2023-02-01 00:12:28,13m 18s


In [12]:
kilometers_per_mile = 1.61
(
    df
    .select([
        'VendorID',
        pl.col('trip_distance').name.suffix('_in_miles'),
        (pl.col('trip_distance') * kilometers_per_mile).name.suffix('_in_km')
    ])
    .head()
)

VendorID,trip_distance_in_miles,trip_distance_in_km
i32,f64,f64
1,0.3,0.483
2,0.0,0.0
2,0.0,0.0
1,18.8,30.268
2,3.22,5.1842


In [16]:
(
    df
    .select([
        pl.col('trip_distance').min().name.suffix('_min'),
        pl.col('trip_distance').max().name.suffix('_max'),
    ])
)

trip_distance_min,trip_distance_max
f64,f64
0.0,335004.33


# Query Statement - **.filter()**

In [20]:
df.describe()

statistic,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
str,f64,str,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",2.913955e6,"""2913955""","""2913955""",2.837138e6,2.913955e6,2.837138e6,"""2837138""",2.913955e6,2.913955e6,2.913955e6,2.913955e6,2.913955e6,2.913955e6,2.913955e6,2.913955e6,2.913955e6,2.913955e6,2.837138e6,2.837138e6
"""null_count""",0.0,"""0""","""0""",76817.0,0.0,76817.0,"""76817""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76817.0,76817.0
"""mean""",1.731282,"""2023-02-15 00:38:01.639923""","""2023-02-15 00:54:02.594501""",1.352847,3.868058,1.514554,null,166.256973,164.231667,1.182896,18.220381,1.560472,0.487875,3.384825,0.511398,0.981305,26.898484,2.285299,0.096778
"""std""",0.452495,null,null,0.889127,268.427268,6.633395,null,64.132296,69.754056,0.529722,17.498195,1.792708,0.100619,3.838069,2.024488,0.18731,21.841097,0.760908,0.338827
"""min""",1.0,"""2008-12-31 23:05:06""","""2008-12-31 23:31:24""",0.0,0.0,1.0,"""N""",1.0,1.0,0.0,-754.05,-7.5,-0.5,-93.0,-57.0,-1.0,-757.55,-2.5,-1.25
"""25%""",1.0,"""2023-02-08 08:43:14""","""2023-02-08 08:59:34""",1.0,1.06,1.0,null,132.0,113.0,1.0,8.6,0.0,0.5,1.0,0.0,1.0,15.48,2.5,0.0
"""50%""",2.0,"""2023-02-14 22:33:21""","""2023-02-14 22:48:15""",1.0,1.79,1.0,null,162.0,162.0,1.0,12.8,1.0,0.5,2.8,0.0,1.0,20.2,2.5,0.0
"""75%""",2.0,"""2023-02-22 08:34:18""","""2023-02-22 08:50:06""",1.0,3.3,1.0,null,234.0,234.0,1.0,20.3,2.5,0.5,4.2,0.0,1.0,28.7,2.5,0.0
"""max""",6.0,"""2023-03-07 13:01:28""","""2023-03-07 13:11:17""",9.0,335004.33,99.0,"""Y""",265.0,265.0,4.0,2203.1,20.8,4.0,482.9,109.9,1.0,2208.1,2.75,1.25


In [37]:
# total null values in passenger_count
null_cnt = (
    df
    .select('passenger_count')
    .null_count()
)
print(null_cnt)

# not null vals, total vals, another way to calc total rows
(
    df
    .select([
        pl.col('passenger_count').count().alias('cnt_without_null'),
        pl.col('passenger_count').len().alias('total_cnt'),
        (pl.col('passenger_count').count() + null_cnt).alias('another_way_to_calc_total_rows')
    ])
)

shape: (1, 1)
┌─────────────────┐
│ passenger_count │
│ ---             │
│ u32             │
╞═════════════════╡
│ 76817           │
└─────────────────┘


cnt_without_null,total_cnt,another_way_to_calc_total_rows
u32,u32,u32
2837138,2913955,2913955


# Fraction of records having zero count passengers.

In [41]:
(
    df
    .select([
        pl.col('passenger_count').eq(0).sum().alias('zero_passenger_count_sum'),
        pl.col('passenger_count').count().alias('zero_passenger_count'),
        ((pl.col('passenger_count').eq(0).sum() / pl.col('passenger_count').count()) * 100).alias('percentage_of_zero_cnt_passengers')
    ])
)

zero_passenger_count_sum,zero_passenger_count,percentage_of_zero_cnt_passengers
u32,u32,f64
47277,2837138,1.666362


# Filtering the passengers where passenger_count = 0

In [43]:
(
    df
    .filter(pl.col('passenger_count').gt(0)).
    select([
        'VendorID',
        'passenger_count'
    ])
)

VendorID,passenger_count
i32,i64
1,2
2,1
2,1
2,1
1,1
…,…
2,1
2,2
2,1


# Using Conditional Operators using **.filter()**
- & and |
- and_() & or_()

In [44]:
(
    df
    .filter(
        pl.col('passenger_count').gt(3) &
        pl.col('trip_distance').gt(100)
    )
)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-02-11 00:00:54,2023-02-11 02:08:12,4,107.49,5,"""N""",132,265,2,400.0,0.0,0.0,0.0,0.0,1.0,402.25,0.0,1.25


In [45]:
(
    df
    .filter(
        pl.col('passenger_count').gt(3) |
        pl.col('trip_distance').gt(100)
    )
    
)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-02-01 00:06:00,2023-02-01 00:10:31,5,1.03,1,"""N""",249,90,1,7.2,1.0,0.5,3.05,0.0,1.0,15.25,2.5,0.0
2,2023-02-01 00:31:02,2023-02-01 00:41:26,5,2.06,1,"""N""",234,4,1,12.1,1.0,0.5,3.42,0.0,1.0,20.52,2.5,0.0
2,2023-02-01 00:47:56,2023-02-01 00:53:02,5,0.93,1,"""N""",114,125,1,7.2,1.0,0.5,0.0,0.0,1.0,12.2,2.5,0.0
2,2023-02-01 00:03:51,2023-02-01 00:04:11,4,0.0,5,"""N""",22,22,1,55.0,0.0,0.0,5.0,0.0,1.0,61.0,0.0,0.0
2,2023-02-01 00:47:14,2023-02-01 00:57:18,4,1.8,1,"""N""",237,229,2,12.1,1.0,0.5,0.0,0.0,1.0,17.1,2.5,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,2023-02-25 12:02:00,2023-02-25 12:23:00,null,14429.87,null,null,261,230,0,27.74,0.0,0.5,6.35,0.0,1.0,38.09,null,null
2,2023-02-26 00:33:00,2023-02-26 00:42:00,null,14235.91,null,null,137,90,0,13.91,0.0,0.5,2.59,0.0,1.0,20.5,null,null
2,2023-02-27 20:04:00,2023-02-27 20:50:00,null,6356.98,null,null,68,226,0,27.05,0.0,0.5,6.21,0.0,1.0,37.26,null,null


In [48]:
(
    df
    .filter(
        pl.col('passenger_count').gt(3) 
        .and_(pl.col('trip_distance').gt(100))
    )
)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-02-11 00:00:54,2023-02-11 02:08:12,4,107.49,5,"""N""",132,265,2,400.0,0.0,0.0,0.0,0.0,1.0,402.25,0.0,1.25


In [49]:
(
    df
    .filter(
        pl.col('passenger_count').gt(3) 
        .or_(pl.col('trip_distance').gt(100))
    )
)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-02-01 00:06:00,2023-02-01 00:10:31,5,1.03,1,"""N""",249,90,1,7.2,1.0,0.5,3.05,0.0,1.0,15.25,2.5,0.0
2,2023-02-01 00:31:02,2023-02-01 00:41:26,5,2.06,1,"""N""",234,4,1,12.1,1.0,0.5,3.42,0.0,1.0,20.52,2.5,0.0
2,2023-02-01 00:47:56,2023-02-01 00:53:02,5,0.93,1,"""N""",114,125,1,7.2,1.0,0.5,0.0,0.0,1.0,12.2,2.5,0.0
2,2023-02-01 00:03:51,2023-02-01 00:04:11,4,0.0,5,"""N""",22,22,1,55.0,0.0,0.0,5.0,0.0,1.0,61.0,0.0,0.0
2,2023-02-01 00:47:14,2023-02-01 00:57:18,4,1.8,1,"""N""",237,229,2,12.1,1.0,0.5,0.0,0.0,1.0,17.1,2.5,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,2023-02-25 12:02:00,2023-02-25 12:23:00,null,14429.87,null,null,261,230,0,27.74,0.0,0.5,6.35,0.0,1.0,38.09,null,null
2,2023-02-26 00:33:00,2023-02-26 00:42:00,null,14235.91,null,null,137,90,0,13.91,0.0,0.5,2.59,0.0,1.0,20.5,null,null
2,2023-02-27 20:04:00,2023-02-27 20:50:00,null,6356.98,null,null,68,226,0,27.05,0.0,0.5,6.21,0.0,1.0,37.26,null,null


# Query Statement - **sort()**

In [50]:
(
    df
    .select([
        'trip_distance',
        'fare_amount'
    ])
    .sort('trip_distance',descending=True)
)

trip_distance,fare_amount
f64,f64
335004.33,17.75
182444.33,12.6
143926.36,105.99
107564.66,24.9
97746.76,19.95
…,…
0.0,6.83
0.0,13.64
0.0,9.76


# sort() + filter()

In [51]:
(
    df
    .filter(
        pl.col('trip_distance').gt(0)
    )
    .sort(
        'trip_distance',descending = True
    )
    .select([
        'trip_distance',
        'fare_amount'
    ])
)

trip_distance,fare_amount
f64,f64
335004.33,17.75
182444.33,12.6
143926.36,105.99
107564.66,24.9
97746.76,19.95
…,…
0.01,14.21
0.01,12.9
0.01,22.78
